**Import requiered libraries**

In [ ]:
pip install evaluate

In [ ]:
from huggingface_hub import login
from datasets import load_dataset
from kaggle_secrets import UserSecretsClient
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import wandb
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

**Load data from Hugging face**

In [3]:
secret_label = "secret_val_0"
secret_value = UserSecretsClient().get_secret(secret_label)
login(secret_value)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
dataset = load_dataset("OmidAghili/Sentiment_Analysis_Train-Validation")
testDataset = load_dataset("OmidAghili/Sentiment_Analysis_Test")

In [5]:
dataset=concatenate_datasets([dataset['train'],dataset['validation']]).shuffle(seed=42)

In [6]:
dataset

Dataset({
    features: ['reviewText', 'label'],
    num_rows: 380945
})

In [7]:
testDataset

DatasetDict({
    train: Dataset({
        features: ['reviewText'],
        num_rows: 20000
    })
})

**Model**

In [8]:
wandb.init(project="sentiment_analysis")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["reviewText"], truncation=True)

tokenized = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")


In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)
        

id2label = {0: "EXTREMELY NEGATIVE", 1: "NEGATIVE",2:"NEURAL",3:"POSITIVE", 4:"EXTREMELY POSITIVE"}
label2id = {"EXTREMELY NEGATIVE": 0, "NEGATIVE": 1,"NEURAL":2,"POSITIVE":3, "EXTREMELY POSITIVE":4}


model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=5, id2label=id2label, label2id=label2id
)

training_args = TrainingArguments(
    output_dir="Sentiment_Analysis_FinalModel",
    learning_rate=2e-4,
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="wandb" 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    eval_dataset=tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub("OmidAghili/Amazon_Reviews")